<style>
body {
    max-width: 900px;
    margin: 40px auto;
    padding: 0 20px;
    font-family: "Georgia", serif;
    line-height: 1.6;
}
</style>

<div style="text-align: center; padding: 60px 60px">
  <h1 style="font-weight: bold; font-size: 3.1em">
    ENCONTRAR MODELOS QUE CUMPLAN LOS SUPUESTOS DE PMDARIMA
  </h1>
</div>

Se busca encontrar un modelo que cumpla los supuestos con los resultados obtenidos en `02-Encontrar-los-Mejores-Modelos`

**RESULTADOS:** Ningun modelo propuesto cumple con independencia en los errores.

In [1]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from verificacion_de_supuestos import * # Una funcion que hice para verificar si los parametros son estacionarios/invertibles
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import ttest_1samp
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.tools.tools import add_constant
from scipy.stats import jarque_bera
from statsmodels.stats.diagnostic import lilliefors

## **CARGAR LOS DATOS**

In [2]:
import pandas as pd
import numpy as np

data=pd.read_csv('MXN00021035.csv')

pre=data.iloc[:,6]  # Precipitacion, es la columna 5
date=data.iloc[:,5] # Date, es la columna 6
date = date.astype(str).str.replace(r'(\d{4})(\d{2})', r'\1/\2', regex=True)    # La fecha está como 195210 y la pasamos a 1952/10 
date = pd.to_datetime(date, format='%Y/%m')                                     # Lo convertimos en fecha
pre = pd.Series(pre.values, index=date)                                         # Creamos una Serie

# Partir la serie para train y test
pre_total = pre.copy()          # Copia de la serie original

# Todas hasta los ultimos 12 meses
pre = pre_total[:-12]           # Entrenamiento: todos menos los últimos 12 meses

from scipy.stats import zscore

# Estandarizar la serie original
zpre = zscore(pre)

# **MODELADO**

## **1 MODELO**

In [3]:
modelo=SARIMAX(zpre,
               order=(1,0,0),
               seasonal_order=(0,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(1, 0, 0)x(0, 1, [1], 12)   Log Likelihood                -711.312
Date:                              Sat, 26 Apr 2025   AIC                           1428.625
Time:                                      21:30:20   BIC                           1442.046
Sample:                                           0   HQIC                          1433.831
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1664      0.031      5.429      0.000       0.106       0.227
ma.S.L12      -0.6853      0.024    -28.483      0.000      -0.732      -0.638
sigma2         0.5198      0.019     26.782      0.000       0.482       0.558
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               224.17
Prob(Q):                              0.94   Prob(JB):                         0.00
Heteroskedasticity (H):               0.89   Skew:                             0.52
Prob(H) (two-sided):                  0.38   Kurtosis:                         5.69
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [4]:
acorr_ljungbox(modelo.resid, period=12, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.089440,NaN
2,0.237416,NaN
3,1.631794,2.014550e-01
4,7.052214,2.941922e-02
5,26.447884,7.684711e-06
6,41.562372,2.055546e-08
7,42.006166,5.873509e-08
8,42.940392,1.198548e-07
9,44.401254,1.786877e-07
10,47.857300,1.051990e-07


## **2 MODELO**

In [5]:
modelo=SARIMAX(zpre,
               order=(0,0,1),
               seasonal_order=(0,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  660
Model:             SARIMAX(0, 0, 1)x(0, 1, 1, 12)   Log Likelihood                -711.750
Date:                            Sat, 26 Apr 2025   AIC                           1429.501
Time:                                    21:30:20   BIC                           1442.923
Sample:                                         0   HQIC                          1434.708
                                            - 660                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1587      0.031      5.089      0.000       0.098       0.220
ma.S.L12      -0.6817      0.024    -28.359      0.000      -0.729      -0.635
sigma2         0.5206      0.020     26.395      0.000       0.482       0.559
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               221.04
Prob(Q):                              0.90   Prob(JB):                         0.00
Heteroskedasticity (H):               0.89   Skew:                             0.53
Prob(H) (two-sided):                  0.38   Kurtosis:                         5.66
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [6]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.249548,NaN
2,1.220081,NaN
3,2.367515,1.238841e-01
4,8.148287,1.700677e-02
5,27.595017,4.417065e-06
6,43.398969,8.551645e-09
7,43.950650,2.370272e-08
8,44.954122,4.779486e-08
9,46.572908,6.761646e-08
10,50.041325,4.012706e-08


## **3 MODELO**

In [7]:
modelo=SARIMAX(zpre,
               order=(1,0,1),
               seasonal_order=(0,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  660
Model:             SARIMAX(1, 0, 1)x(0, 1, 1, 12)   Log Likelihood                -711.177
Date:                            Sat, 26 Apr 2025   AIC                           1430.355
Time:                                    21:30:21   BIC                           1448.250
Sample:                                         0   HQIC                          1437.297
                                            - 660                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.2786      0.217      1.283      0.199      -0.147       0.704
ma.L1         -0.1149      0.223     -0.515      0.606      -0.552       0.322
ma.S.L12      -0.6862      0.024    -28.283      0.000      -0.734      -0.639
sigma2         0.5196      0.020     26.467      0.000       0.481       0.558
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               228.74
Prob(Q):                              1.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.88   Skew:                             0.51
Prob(H) (two-sided):                  0.36   Kurtosis:                         5.72
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [8]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.136052,NaN
2,0.136083,NaN
3,1.431875,NaN
4,6.727948,9.491374e-03
5,26.445328,1.809131e-06
6,41.408615,5.356129e-09
7,41.816104,1.821177e-08
8,42.679221,4.291645e-08
9,44.080115,7.126546e-08
10,47.652562,4.163102e-08


## **4 MODELO**

In [9]:
modelo=SARIMAX(zpre,
               order=(2,0,0),
               seasonal_order=(0,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(2, 0, 0)x(0, 1, [1], 12)   Log Likelihood                -711.190
Date:                              Sat, 26 Apr 2025   AIC                           1430.380
Time:                                      21:30:21   BIC                           1448.275
Sample:                                           0   HQIC                          1437.322
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1636      0.031      5.226      0.000       0.102       0.225
ar.L2          0.0195      0.038      0.519      0.604      -0.054       0.093
ma.S.L12      -0.6863      0.024    -28.364      0.000      -0.734      -0.639
sigma2         0.5196      0.020     26.373      0.000       0.481       0.558
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               229.13
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               0.88   Skew:                             0.52
Prob(H) (two-sided):                  0.36   Kurtosis:                         5.72
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [10]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.133046,NaN
2,0.133993,NaN
3,1.535713,NaN
4,6.767412,9.283738e-03
5,26.475379,1.782151e-06
6,41.376277,5.441405e-09
7,41.780984,1.851953e-08
8,42.645970,4.358726e-08
9,44.023848,7.312031e-08
10,47.591496,4.279022e-08


## **5 MODELO**

In [11]:
modelo=SARIMAX(zpre,
               order=(1,0,0),
               seasonal_order=(0,1,2,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                        SARIMAX Results                                        
===============================================================================================
Dep. Variable:                                       y   No. Observations:                  660
Model:             SARIMAX(1, 0, 0)x(0, 1, [1, 2], 12)   Log Likelihood                -711.297
Date:                                 Sat, 26 Apr 2025   AIC                           1430.594
Time:                                         21:30:22   BIC                           1448.489
Sample:                                              0   HQIC                          1437.536
                                                 - 660                                         
Covariance Type:                                   opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1669      0.031      5.420      0.000       0.107       0.227
ma.S.L12      -0.6800      0.028    -24.159      0.000      -0.735      -0.625
ma.S.L24      -0.0076      0.030     -0.256      0.798      -0.066       0.051
sigma2         0.5198      0.019     26.703      0.000       0.482       0.558
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               225.97
Prob(Q):                              0.94   Prob(JB):                         0.00
Heteroskedasticity (H):               0.89   Skew:                             0.52
Prob(H) (two-sided):                  0.37   Kurtosis:                         5.70
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [12]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.088677,NaN
2,0.234900,NaN
3,1.614950,NaN
4,7.062003,7.873591e-03
5,26.443091,1.811156e-06
6,41.545503,5.009686e-09
7,41.991504,1.674941e-08
8,42.926501,3.824015e-08
9,44.380290,6.213357e-08
10,47.864762,3.784053e-08


## **6 MODELO**

In [13]:
modelo=SARIMAX(zpre,
               order=(1,0,0),
               seasonal_order=(1,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(1, 0, 0)x(1, 1, [1], 12)   Log Likelihood                -711.300
Date:                              Sat, 26 Apr 2025   AIC                           1430.599
Time:                                      21:30:22   BIC                           1448.495
Sample:                                           0   HQIC                          1437.541
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1669      0.031      5.416      0.000       0.106       0.227
ar.S.L12       0.0092      0.044      0.212      0.832      -0.076       0.095
ma.S.L12      -0.6901      0.038    -18.391      0.000      -0.764      -0.617
sigma2         0.5198      0.019     26.700      0.000       0.482       0.558
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               225.66
Prob(Q):                              0.94   Prob(JB):                         0.00
Heteroskedasticity (H):               0.89   Skew:                             0.52
Prob(H) (two-sided):                  0.38   Kurtosis:                         5.70
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [14]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.088749,NaN
2,0.235282,NaN
3,1.617918,NaN
4,7.060476,7.880304e-03
5,26.444536,1.809848e-06
6,41.549316,5.000364e-09
7,41.994892,1.672235e-08
8,42.929681,3.818345e-08
9,44.384597,6.201138e-08
10,47.864418,3.784639e-08


## **7 MODELO**

In [15]:
modelo=SARIMAX(zpre,
               order=(0,0,2),
               seasonal_order=(0,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                       
============================================================================================
Dep. Variable:                                    y   No. Observations:                  660
Model:             SARIMAX(0, 0, 2)x(0, 1, [1], 12)   Log Likelihood                -711.446
Date:                              Sat, 26 Apr 2025   AIC                           1430.893
Time:                                      21:30:23   BIC                           1448.788
Sample:                                           0   HQIC                          1437.835
                                              - 660                                         
Covariance Type:                                opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1601      0.031      5.129      0.000       0.099       0.221
ma.L2          0.0320      0.037      0.871      0.384      -0.040       0.104
ma.S.L12      -0.6846      0.024    -28.424      0.000      -0.732      -0.637
sigma2         0.5201      0.020     26.248      0.000       0.481       0.559
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               228.40
Prob(Q):                              0.95   Prob(JB):                         0.00
Heteroskedasticity (H):               0.89   Skew:                             0.52
Prob(H) (two-sided):                  0.37   Kurtosis:                         5.71
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [16]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.189177,NaN
2,0.250878,NaN
3,1.946468,NaN
4,7.354676,6.688856e-03
5,27.081362,1.316306e-06
6,42.131667,3.762123e-09
7,42.588989,1.259262e-08
8,43.498457,2.927886e-08
9,44.891941,4.917391e-08
10,48.450335,2.907012e-08


## **8 MODELO**

In [17]:
modelo=SARIMAX(zpre,
               order=(0,0,1),
               seasonal_order=(0,1,2,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                        SARIMAX Results                                        
===============================================================================================
Dep. Variable:                                       y   No. Observations:                  660
Model:             SARIMAX(0, 0, 1)x(0, 1, [1, 2], 12)   Log Likelihood                -711.737
Date:                                 Sat, 26 Apr 2025   AIC                           1431.474
Time:                                         21:30:24   BIC                           1449.369
Sample:                                              0   HQIC                          1438.416
                                                 - 660                                         
Covariance Type:                                   opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1591      0.031      5.059      0.000       0.097       0.221
ma.S.L12      -0.6766      0.028    -24.012      0.000      -0.732      -0.621
ma.S.L24      -0.0071      0.030     -0.238      0.812      -0.066       0.052
sigma2         0.5206      0.020     26.342      0.000       0.482       0.559
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               222.62
Prob(Q):                              0.90   Prob(JB):                         0.00
Heteroskedasticity (H):               0.89   Skew:                             0.53
Prob(H) (two-sided):                  0.37   Kurtosis:                         5.67
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [18]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.248530,NaN
2,1.220950,NaN
3,2.355162,NaN
4,8.163010,4.275335e-03
5,27.595911,1.017710e-06
6,43.391509,2.032262e-09
7,43.945926,6.583858e-09
8,44.950458,1.484917e-08
9,46.564274,2.285474e-08
10,50.047333,1.413906e-08


## **9 MODELO**

In [19]:
modelo=SARIMAX(zpre,
               order=(0,0,1),
               seasonal_order=(1,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  660
Model:             SARIMAX(0, 0, 1)x(1, 1, 1, 12)   Log Likelihood                -711.739
Date:                            Sat, 26 Apr 2025   AIC                           1431.479
Time:                                    21:30:24   BIC                           1449.374
Sample:                                         0   HQIC                          1438.421
                                            - 660                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1591      0.031      5.055      0.000       0.097       0.221
ar.S.L12       0.0086      0.044      0.196      0.844      -0.078       0.095
ma.S.L12      -0.6862      0.038    -18.139      0.000      -0.760      -0.612
sigma2         0.5206      0.020     26.341      0.000       0.482       0.559
===================================================================================
Ljung-Box (L1) (Q):                   0.01   Jarque-Bera (JB):               222.35
Prob(Q):                              0.90   Prob(JB):                         0.00
Heteroskedasticity (H):               0.89   Skew:                             0.53
Prob(H) (two-sided):                  0.37   Kurtosis:                         5.67
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [20]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.248640,NaN
2,1.220866,NaN
3,2.357464,NaN
4,8.160667,4.280864e-03
5,27.596367,1.017478e-06
6,43.393730,2.030056e-09
7,43.947631,6.578490e-09
8,44.951912,1.483907e-08
9,46.566545,2.283093e-08
10,50.047268,1.413948e-08


## **10 MODELO**

In [21]:
modelo=SARIMAX(zpre,
               order=(2,0,1),
               seasonal_order=(0,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  660
Model:             SARIMAX(2, 0, 1)x(0, 1, 1, 12)   Log Likelihood                -711.175
Date:                            Sat, 26 Apr 2025   AIC                           1432.349
Time:                                    21:30:25   BIC                           1454.719
Sample:                                         0   HQIC                          1441.027
                                            - 660                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.2492      1.981      0.126      0.900      -3.633       4.132
ar.L2          0.0063      0.333      0.019      0.985      -0.647       0.659
ma.L1         -0.0859      1.977     -0.043      0.965      -3.960       3.788
ma.S.L12      -0.6862      0.024    -28.108      0.000      -0.734      -0.638
sigma2         0.5196      0.020     26.300      0.000       0.481       0.558
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               229.20
Prob(Q):                              1.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.88   Skew:                             0.51
Prob(H) (two-sided):                  0.36   Kurtosis:                         5.73
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [22]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.140982,NaN
2,0.142544,NaN
3,1.458317,NaN
4,6.724591,NaN
5,26.463652,2.685439e-07
6,41.402516,1.022252e-09
7,41.807146,4.408568e-09
8,42.665866,1.213866e-08
9,44.056261,2.256104e-08
10,47.636825,1.396424e-08


## **11 MODELO**

In [23]:
modelo=SARIMAX(zpre,
               order=(0,0,0),
               seasonal_order=(0,1,1,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 SARIMAX Results                                  
==================================================================================
Dep. Variable:                          y   No. Observations:                  660
Model:             SARIMAX(0, 1, [1], 12)   Log Likelihood                -719.797
Date:                    Sat, 26 Apr 2025   AIC                           1443.593
Time:                            21:30:26   BIC                           1452.541
Sample:                                 0   HQIC                          1447.064
                                    - 660                                         
Covariance Type:                      opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.S.L12      -0.6533      0.025    -26.057      0.000      -0.702      -0.604
sigma2         0.5344      0.019     28.147      0.000       0.497       0.572
===================================================================================
Ljung-Box (L1) (Q):                  15.79   Jarque-Bera (JB):               286.71
Prob(Q):                              0.00   Prob(JB):                         0.00
Heteroskedasticity (H):               0.83   Skew:                             0.59
Prob(H) (two-sided):                  0.17   Kurtosis:                         6.04
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [24]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,18.383847,NaN
2,19.527333,9.917048e-06
3,20.310714,3.886730e-05
4,27.528968,4.560262e-06
5,53.592080,6.406348e-11
6,73.402658,2.003677e-14
7,74.883915,4.055452e-14
8,75.550613,1.108538e-13
9,77.253111,1.743214e-13
10,83.190537,3.746241e-14


## **12 MODELO**

In [25]:
modelo=SARIMAX(zpre,
               order=(1,0,0),
               seasonal_order=(1,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  660
Model:             SARIMAX(1, 0, 0)x(1, 1, 0, 12)   Log Likelihood                -757.585
Date:                            Sat, 26 Apr 2025   AIC                           1521.170
Time:                                    21:30:26   BIC                           1534.592
Sample:                                         0   HQIC                          1526.377
                                            - 660                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.1071      0.030      3.620      0.000       0.049       0.165
ar.S.L12      -0.4380      0.025    -17.588      0.000      -0.487      -0.389
sigma2         0.6043      0.022     27.801      0.000       0.562       0.647
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               268.10
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               0.85   Skew:                             0.25
Prob(H) (two-sided):                  0.23   Kurtosis:                         6.11
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [26]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.059138,NaN
2,0.063230,NaN
3,1.656154,1.981234e-01
4,4.652222,9.767485e-02
5,16.056469,1.104152e-03
6,21.892503,2.105402e-04
7,22.475281,4.251332e-04
8,22.477668,9.916909e-04
9,23.468955,1.411806e-03
10,26.313494,9.280917e-04


## **13 MODELO**

In [27]:
modelo=SARIMAX(zpre,
               order=(0,0,1),
               seasonal_order=(1,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  660
Model:             SARIMAX(0, 0, 1)x(1, 1, [], 12)   Log Likelihood                -757.624
Date:                             Sat, 26 Apr 2025   AIC                           1521.248
Time:                                     21:30:26   BIC                           1534.670
Sample:                                          0   HQIC                          1526.455
                                             - 660                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.1066      0.030      3.579      0.000       0.048       0.165
ar.S.L12      -0.4376      0.025    -17.550      0.000      -0.487      -0.389
sigma2         0.6044      0.022     27.506      0.000       0.561       0.647
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               263.42
Prob(Q):                              0.99   Prob(JB):                         0.00
Heteroskedasticity (H):               0.85   Skew:                             0.25
Prob(H) (two-sided):                  0.23   Kurtosis:                         6.08
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [28]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.073572,NaN
2,0.185384,NaN
3,1.693571,1.931310e-01
4,4.768123,9.217544e-02
5,16.035395,1.115193e-03
6,21.965430,2.036211e-04
7,22.512569,4.182314e-04
8,22.514222,9.766213e-04
9,23.580028,1.350007e-03
10,26.311277,9.289049e-04


## **14 MODELO**

In [29]:
modelo=SARIMAX(zpre,
               order=(0,0,1),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                  660
Model:             SARIMAX(0, 0, 1)x(0, 1, [], 12)   Log Likelihood                -823.425
Date:                             Sat, 26 Apr 2025   AIC                           1650.849
Time:                                     21:30:26   BIC                           1659.797
Sample:                                          0   HQIC                          1654.321
                                             - 660                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.0875      0.030      2.927      0.003       0.029       0.146
sigma2         0.7434      0.024     30.671      0.000       0.696       0.791
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               416.77
Prob(Q):                              0.98   Prob(JB):                         0.00
Heteroskedasticity (H):               0.81   Skew:                            -0.22
Prob(H) (two-sided):                  0.11   Kurtosis:                         6.90
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [30]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.013171,NaN
2,0.063025,8.017779e-01
3,0.701844,7.040385e-01
4,3.527965,3.171533e-01
5,9.965172,4.101846e-02
6,12.316652,3.069731e-02
7,13.800257,3.194876e-02
8,13.876051,5.343034e-02
9,14.677597,6.572600e-02
10,16.871688,5.076365e-02


## **15 MODELO**

In [31]:
modelo=SARIMAX(zpre,
               order=(1,0,0),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                     SARIMAX Results                                      
==========================================================================================
Dep. Variable:                                  y   No. Observations:                  660
Model:             SARIMAX(1, 0, 0)x(0, 1, 0, 12)   Log Likelihood                -823.492
Date:                            Sat, 26 Apr 2025   AIC                           1650.985
Time:                                    21:30:26   BIC                           1659.932
Sample:                                         0   HQIC                          1654.456
                                            - 660                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0849      0.030      2.848      0.004       0.026       0.143
sigma2         0.7436      0.024     30.948      0.000       0.697       0.791
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):               424.32
Prob(Q):                              0.97   Prob(JB):                         0.00
Heteroskedasticity (H):               0.80   Skew:                            -0.22
Prob(H) (two-sided):                  0.11   Kurtosis:                         6.94
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [32]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,0.030665,NaN
2,0.189509,6.633258e-01
3,0.846695,6.548512e-01
4,3.642731,3.027167e-01
5,10.189362,3.735589e-02
6,12.516573,2.835568e-02
7,14.021648,2.939527e-02
8,14.101334,4.940811e-02
9,14.867482,6.177399e-02
10,17.320001,4.393434e-02


## **16 MODELO**

In [33]:
modelo=SARIMAX(zpre,
               order=(0,0,0),
               seasonal_order=(0,1,0,12)).fit()

modelo.summary()

c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\ProgramData\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


<class 'statsmodels.iolib.summary.Summary'>
"""
                                SARIMAX Results                                 
================================================================================
Dep. Variable:                        y   No. Observations:                  660
Model:             SARIMAX(0, 1, 0, 12)   Log Likelihood                -825.836
Date:                  Sat, 26 Apr 2025   AIC                           1653.671
Time:                          21:30:26   BIC                           1658.145
Sample:                               0   HQIC                          1655.407
                                  - 660                                         
Covariance Type:                    opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
sigma2         0.7490      0.024     31.151      0.000       0.702       0.796
===================================================================================
Ljung-Box (L1) (Q):                   4.70   Jarque-Bera (JB):               432.92
Prob(Q):                              0.03   Prob(JB):                         0.00
Heteroskedasticity (H):               0.77   Skew:                            -0.17
Prob(H) (two-sided):                  0.06   Kurtosis:                         6.99
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [34]:
acorr_ljungbox(modelo.resid, period=24, model_df=len(modelo.params)-1)

,lb_stat,lb_pvalue
1,5.277237,2.160601e-02
2,5.300793,7.062320e-02
3,5.719894,1.260623e-01
4,8.943236,6.253408e-02
5,16.789448,4.916919e-03
6,19.507794,3.386705e-03
7,20.734564,4.183529e-03
8,20.931159,7.332566e-03
9,21.965562,8.988763e-03
10,24.939913,5.460571e-03
